In [1]:
import glob
import pandas as pd
import re
import nilearn as nil
from nilearn import *
from nilearn import image
from os.path import basename
import numpy as np

get all series

get masks

iterate through series

iterate through masks

produce the vector for this mask/series

for each trial [CS, CG, FS, FG], identify the moment the subject learns the class of the trial;
this may be slightly different for Go and Stop trials. For Stop trials it is simply the moment of the tone;
for Go trials it's harder to specify.

for each TRIAL, you will end up with a data table with columns (a) time from trial class reveal to image; (b) separate columns for measurement of each ROI
these can be concatenated across trials, runs, and subjects

then create n_{ROI}*n_{trial classes} graphs of samples, where x is the time from trial class reveal to image, and y is the measurement. plot a lowess curve.

potentially overlay the lowess curves for each trial type so that the response within each ROI can be easily compared.


In [2]:
#need to replace these with references to the config yml
dropbox_data_dir = '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/'
fmriprep_dir = '/Users/benjaminsmith/Google Drive/oregon/data/DEV/bids_data/derivatives/fmriprep/'
nii_raw_path = 'sub-DEV*/ses-wave1/func/s6_sub-DEV*_ses-wave1_task-SST_acq-1_bold_space-MNI152NLin2009cAsym_preproc.nii'
mask_location = '/Users/benjaminsmith/Google Drive/oregon/data/DEV/brainmaps/striatum/'

get all series

In [3]:
glob_path = fmriprep_dir + nii_raw_path
print("looking up " + glob_path)
nii_raw_files = glob.glob(glob_path)

looking up /Users/benjaminsmith/Google Drive/oregon/data/DEV/bids_data/derivatives/fmriprep/sub-DEV*/ses-wave1/func/s6_sub-DEV*_ses-wave1_task-SST_acq-1_bold_space-MNI152NLin2009cAsym_preproc.nii


In [4]:
print(nii_raw_files)

['/Users/benjaminsmith/Google Drive/oregon/data/DEV/bids_data/derivatives/fmriprep/sub-DEV285/ses-wave1/func/s6_sub-DEV285_ses-wave1_task-SST_acq-1_bold_space-MNI152NLin2009cAsym_preproc.nii', '/Users/benjaminsmith/Google Drive/oregon/data/DEV/bids_data/derivatives/fmriprep/sub-DEV004/ses-wave1/func/s6_sub-DEV004_ses-wave1_task-SST_acq-1_bold_space-MNI152NLin2009cAsym_preproc.nii', '/Users/benjaminsmith/Google Drive/oregon/data/DEV/bids_data/derivatives/fmriprep/sub-DEV293/ses-wave1/func/s6_sub-DEV293_ses-wave1_task-SST_acq-1_bold_space-MNI152NLin2009cAsym_preproc.nii']


get masks

In [5]:
mask_paths = glob.glob(mask_location+"*.nii.gz")
mask_labels = [re.match(".*/(.*)\.nii\.gz",mp)[1] for mp in mask_paths]

mask_df = pd.DataFrame({
    'mask_label':mask_labels,
    'mask_path': mask_paths
})

iterate through series

In [7]:
#mark the number of seconds each scan takes.
TR = 2.0

roi_data = {}

for nii in nii_raw_files:
    series_filename = basename(nii)
    match_data = re.match('.*(DEV\d*)_ses-wave(\d+)*',series_filename)
    dev_name = match_data[1]
    dev_wave = match_data[2]

    #print(basename(nii))
    print(dev_name + ", " + dev_wave)

    active_img = image.load_img(nii)

    #create a template data frame to populate with the ROI data, for this run
    run_len = active_img.shape[3]
    if dev_name not in roi_data.keys():
        roi_data[dev_name]={}
    
    run_df = pd.DataFrame(index=range(0,run_len))
    #run_df = pd.DataFrame({'image_id':range(0,run_len)})
    #run_df['TR_onset']=run_df.image_id*TR


    
    for m_i, m_set in mask_df.iterrows():
        print(m_set['mask_label'])
        print('producing the vector for this mask/series...')
        
        #active_mask = nilearn.masking.compute_brain_mask(m_set['mask_path'])
        mask_raw = nil.image.load_img(m_set['mask_path'])
        mask_in_subj_space = nil.image.resample_img(mask_raw, target_affine=active_img.affine,target_shape = active_img.slicer[:,:,:,0].shape)
        mask_binarized = nil.image.binarize_img(mask_in_subj_space,threshold=50)
        active_img_masked = nil.masking.apply_mask(active_img, mask_binarized)
        activity_vector = active_img_masked.mean(axis=1)
        run_df[m_set['mask_label']]=activity_vector
        print("created an activity vector for this mask with the following length: " + str(len(activity_vector)))
        #
        #nil.plotting.plot_img(mask_applied)
        #nilearn.plotting.view_img(active_mask)
    #now we've gone through all the masks for this run, add the run_df to the dict dict
    roi_data[dev_name][dev_wave] = run_df



        

        



DEV285, 1
harvardoxford-subcortical_prob_Left Putamen
producing the vector for this mask/series...


/Users/benjaminsmith/anaconda/envs/nilearn/lib/python3.8/site-packages/nilearn/image/image.py:1041: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the data has been converted to int32.
  return new_img_like(niimg, result, niimg.affine)


created an activity vector for this mask with the following length: 212
harvardoxford-subcortical_prob_Right Accumbens
producing the vector for this mask/series...


/Users/benjaminsmith/anaconda/envs/nilearn/lib/python3.8/site-packages/nilearn/image/image.py:1041: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the data has been converted to int32.
  return new_img_like(niimg, result, niimg.affine)


created an activity vector for this mask with the following length: 212
harvardoxford-subcortical_prob_Right Putamen
producing the vector for this mask/series...


...

created an activity vector for this mask with the following length: 212
harvardoxford-subcortical_prob_Left Caudate
producing the vector for this mask/series...


/Users/benjaminsmith/anaconda/envs/nilearn/lib/python3.8/site-packages/nilearn/image/image.py:1041: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the data has been converted to int32.
  return new_img_like(niimg, result, niimg.affine)


created an activity vector for this mask with the following length: 212
harvardoxford-subcortical_prob_Right Caudate
producing the vector for this mask/series...


/Users/benjaminsmith/anaconda/envs/nilearn/lib/python3.8/site-packages/nilearn/image/image.py:1041: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the data has been converted to int32.
  return new_img_like(niimg, result, niimg.affine)


created an activity vector for this mask with the following length: 212


In [9]:
for s in roi_data.keys():
    print(s)
    for wave in roi_data[s].keys():
        print(wave)
        print(roi_data[s][wave].shape)
        print(roi_data[s][wave].columns)

DEV285
1
(212, 6)
Index(['harvardoxford-subcortical_prob_Left Putamen',
       'harvardoxford-subcortical_prob_Right Accumbens',
       'harvardoxford-subcortical_prob_Right Putamen',
       'harvardoxford-subcortical_prob_Left Accumbens',
       'harvardoxford-subcortical_prob_Left Caudate',
       'harvardoxford-subcortical_prob_Right Caudate'],
      dtype='object')
DEV004
1
(207, 6)
Index(['harvardoxford-subcortical_prob_Left Putamen',
       'harvardoxford-subcortical_prob_Right Accumbens',
       'harvardoxford-subcortical_prob_Right Putamen',
       'harvardoxford-subcortical_prob_Left Accumbens',
       'harvardoxford-subcortical_prob_Left Caudate',
       'harvardoxford-subcortical_prob_Right Caudate'],
      dtype='object')
DEV293
1
(212, 6)
Index(['harvardoxford-subcortical_prob_Left Putamen',
       'harvardoxford-subcortical_prob_Right Accumbens',
       'harvardoxford-subcortical_prob_Right Putamen',
       'harvardoxford-subcortical_prob_Left Accumbens',
       'harvardo

In [10]:
sst_all_behavioral_data = pd.read_csv(dropbox_data_dir + "sst_behavioral_data_all.csv")



for each trial [CS, CG, FS, FG], identify the moment the subject learns the class of the trial;

In [11]:
# for each trial [CS, CG, FS, FG], identify the moment the subject learns the class of the trial;

# this may be slightly different for Go and Stop trials. For Stop trials it is simply the moment of the tone;
# for Go trials it's harder to specify.

def get_moment_trial_type_revealed(trial_type, ssdr,first_tone):
    trial_type = trial_type.tolist()
    ssdr = ssdr.tolist()
    expected_tone_time = pd.Series([0.]*len(ssdr))
    expected_tone_time[0]=first_tone
    expected_tone_time[1]=first_tone

    for i in range(2,len(expected_tone_time)):
         #these are in increments of 2 because every second event is a cue
         #previous trial was a stop
        if trial_type[i]=="Stop":
            expected_tone_time[i] = ssdr[i]
        else:
            expected_tone_time[i] = expected_tone_time[i-1]
    #print(expected_tone_time)
    return(expected_tone_time)




# but might be the expected time of the onset, which would be the _previous_ stop trial.
#create expected tone table
expected_tone_table = sst_all_behavioral_data.groupby(['subid', 'waveid','runid']).apply(
    lambda sabd_set: get_moment_trial_type_revealed(sabd_set['go_no_go_condition_label'], sabd_set['SSD_recorded'],first_tone=2)
)
expected_tone_table2=expected_tone_table.reset_index()
expected_tone_table2.rename(columns={0:'class_type_reveal'},inplace=True)
expected_tone_table2.loc[:,'trial_n']=expected_tone_table2.level_3+1
expected_tone_table2.drop('level_3',axis=1,inplace=True)

sst_all_behavioral_data = sst_all_behavioral_data.merge(expected_tone_table2,
left_on=['subid','waveid','runid','trial_n'],
right_on =['subid','waveid','runid','trial_n'],
how='left'
    
    )


In [12]:
sst_all_behavioral_data['class_type_reveal_onset'] = sst_all_behavioral_data.class_type_reveal + sst_all_behavioral_data.onset
sst_all_behavioral_data.loc[sst_all_behavioral_data.go_no_go_condition_label=='Cue','class_type_reveal_onset']=None

Now we need a list of the TRs by their time of appearance; that way we can mark which ones are relevant to the ROIs for each trial.

For now, let's assume that the timing of the TRs matches the timing in the matlab files, but this MUST be checked.

In [ ]:
# run_length = 207
# rt_time_list = [x*2.0 for x in range(run_length)]

these are going to be generated dynamically based on the run length

In [18]:
# (is there a way to do this vectorized over ALL the data,
# or is it more sensible to loop through subjects/runs to do it?
# tend to think looping through subjects makes more sense.)
all_subj_df_list = [] 
for s in roi_data.keys():
    print(s)
    for wave in roi_data[s].keys():
        run_length = roi_data[s][wave].shape[0]
        print(s +', ' + wave + ':' + str(run_length))
        rt_time_list = [x*2.0 for x in range(run_length)]
        #now we have a list of the the RTs
        #we have actual data at each data point
        #we need to get the RTs related to each particular trial.
        run_behavioral_data = sst_all_behavioral_data[(sst_all_behavioral_data.subid==s) & (sst_all_behavioral_data.waveid==int(wave))]

        run_trial_df_list = []
        for i, trial in run_behavioral_data.iterrows():
            #print(i)
            t_class_type_reveal_onset = trial['class_type_reveal_onset']
            #print(trial[['trial_n','class_type_reveal_onset']])
            #grab the set of TR times that are relevant based on its
            

            trial_rt_timing_all = pd.DataFrame({'tr':rt_time_list,'offset':[(tr - t_class_type_reveal_onset) for tr in rt_time_list]})
            trs_to_use_for_trial = ((trial_rt_timing_all.offset>-10) & (trial_rt_timing_all.offset<20))
            trial_rt_timing_inscope = trial_rt_timing_all.loc[trs_to_use_for_trial]

            #and we will also want to grab the actual data we already extracted...
            trial_roi_data = pd.merge(trial_rt_timing_inscope,roi_data[s][wave],how='left',
            left_index=True,right_index=True
            )
            trial_roi_data['t_class_type_reveal_onset']=t_class_type_reveal_onset
            trial_roi_data['subid']=s
            trial_roi_data['wave']=wave
            trial_roi_data['go_no_go_condition_label']=trial[['go_no_go_condition_label']][0]
            trial_roi_data['condition']=trial[['condition']][0]
            trial_roi_data['trial_n']=trial[['trial_n']][0]
            trial_roi_data['trial_n']=trial[['trial_n']][0]
            
            run_trial_df_list.append(trial_roi_data)
            #cool. now what? 

        run_trial_df = pd.concat(run_trial_df_list)
        run_trial_df.reset_index(drop=True,inplace=True)

        all_subj_df_list.append(run_trial_df)
all_subj_df = pd.concat(all_subj_df_list)

DEV285
DEV285, 1:212
DEV004
DEV004, 1:207
DEV293
DEV293, 1:212


In [14]:
all_subj_df.groupby(['trial_n','subid','wave']).offset.count()

trial_n  subid   wave
1        DEV004  1       12
         DEV285  1       12
         DEV293  1       12
3        DEV004  1       12
         DEV285  1       13
                         ..
253      DEV285  1        9
         DEV293  1        9
255      DEV004  1        1
         DEV285  1        7
         DEV293  1        7
Name: offset, Length: 384, dtype: int64

In [19]:
all_subj_df.to_csv(dropbox_data_dir + 'SST_roi_by_time_point.csv')

We should have up to 420 or so.

In [15]:
sst_all_behavioral_data.loc[sst_all_behavioral_data.subid=='DEV293'][['trial_n','onset','class_type_reveal_onset','condition']]

,trial_n,onset,class_type_reveal_onset,condition
248832,1,0.011194,2.011194,FailedStop
248833,2,0.000000,NaN,Cue
248834,3,2.763036,4.763036,CorrectGo
248835,4,0.000000,NaN,Cue
248836,5,5.528667,7.528667,CorrectGo
...,...,...,...,...
249083,252,0.000000,NaN,Cue
249084,253,414.584341,414.885800,CorrectGo
249085,254,0.000000,NaN,Cue
249086,255,417.849799,418.151915,CorrectStop


In [ ]:

# for each TRIAL, you will end up with a data table with columns (a) time from trial class reveal to image; (b) separate columns for measurement of each ROI
# these can be concatenated across trials, runs, and subjects

# then create n_{ROI}*n_{trial classes} graphs of samples, where x is the time from trial class reveal to image, and y is the measurement. plot a lowess curve.

# potentially overlay the lowess curves for each trial type so that the response within each ROI can be easily compared.
